# 大数据分析实验1

*环境配置真的烦*

我是用的是hadoop streaming+python的方法实现的，这里为了方便map和reduce写在一起了

## 数据抽样

In [5]:
#import sys

'''
review_id|longitude|latitude|altitude|review_date|temperature|rating|user_id|user_birthday|usr_nationality|user_career|usr_income

user_career

'''

#首先对职业进行计数并计算分层抽样个数 一个Map Reduce
f=open('./data.txt','r')
careers={}
for line in f.readlines():
    line=line.replace('\n','')
    career=line.split('|')[-2]
    #这里输出 career 1 Map阶段结束
    
    #reduce
    if career in careers.keys():
        careers[career]=careers[career]+1
    else:
        careers[career]=1

f.close()
f=open('./career_count.txt','w')
lines=[]
for career in careers.keys():
    line=career+' '+str(int(careers[career]*0.02))+'\n'
    lines.append(line)
lines[-1]=lines[-1].replace('\n','')
f.writelines(lines)
f.close()
    

In [6]:
#进行分层抽样
career_counts={'programmer':12002,'teacher':12150,'farmer':12009,'doctor':11916,'Manager':11910,'accountant':11747,'artist':11803,'writer':12131}

f=open('./data.txt','r')
f1=open('./D_Sample.txt','w')
lines=[]
for line in f.readlines():
    line=line.replace('\n','')
    
    career=line.split('|')[-2]
    if career_counts[career]>0:
        new_line=line+'\n'
        lines.append(new_line)
        career_counts[career]=career_counts[career]-1

f.close()
lines[-1]=lines[-1].replace('\n','')
f1.writelines(lines)
f1.close()

    

## 数据过滤

In [3]:
'''
review_id|longitude|latitude|altitude|review_date|temperature|rating|user_id|user_birthday|usr_nationality|user_career|usr_income

longitude latitude

'''
longitude_min=8.1461259 
latitude_min=56.5824856
longitude_max=11.1993265
latitude_max=57.750511


f=open('./D_Sample.txt','r')
f1=open('./D_Filter.txt','w')
lines=[]
for line in f.readlines():
    line=line.replace('\n','')
    
    longitude=float(line.split('|')[1])
    latitude=float(line.split('|')[2])
    
    if longitude>longitude_min and longitude<longitude_max and latitude>latitude_min and latitude<latitude_max:
        new_line=line+'\n'
        lines.append(new_line)

f.close()
lines[-1]=lines[-1].replace('\n','')
f1.writelines(lines)
f1.close()



## 数据标准化和归一化

In [21]:
'''
review_id|longitude|latitude|altitude|review_date|temperature|rating|user_id|user_birthday|usr_nationality|user_career|usr_income

std:user_birthday review_date temperature

1:rating

'''
# 统计rating最大最小值

f=open('./D_Filter.txt','r',encoding='utf-8')
rating_max=83
rating_min=78
count=1
max_line=1
min_line=1
for line in f.readlines():
    line=line.replace('\n','')
    
    rating=line.split('|')[6]
    if rating!='?':
        rating=float(rating)
        if rating>rating_max:
            max_line=count
            rating_max=rating
        if rating<rating_min:
            rating_min=rating
            min_line=count
    count+=1

f.close()
f=open('./rating_minmax.txt','w')
f.writelines(str(rating_min)+'\n')
f.writelines(str(rating_max))
f.close()

In [4]:
'''
review_id|longitude|latitude|altitude|review_date|temperature|rating|user_id|user_birthday|usr_nationality|user_career|usr_income

std:user_birthday review_date temperature

'''

month_dict={'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}

f=open('./D_Filter.txt','r',encoding='utf-8')
rating_max=101.63
rating_min=-261.13


def time_std(time_):
    if '/' in time_:
        return YMD2Y_M_D(time_)
    elif ',' in time_:
        return MDY2Y_M_D(time_)
    else:
        return time_
    
def tem_std(tem):
    if '℉' in tem:
        return F2C(tem)
    else:
        return tem

def YMD2Y_M_D(ymd):
    [year,month,day]=ymd.split('/')
    return year+'-'+month+'-'+day

def MDY2Y_M_D(mdy):
    month=mdy.split(' ')[0]
    [day,year]=mdy.split(' ')[1].split(',')
    return year+'-'+str(month_dict[month])+'-'+day

def F2C(f_tem):
    ftem=float(f_tem.replace('℉',''))
    ctem=(ftem-32)/1.8
    ctem='%.2f'%ctem
    return str(ctem)+'℃'

def rating_1(rating):
    if rating=='?':
        return rating
    else:
        rating=float(rating)
        rating=(rating-rating_min)/(rating_max-rating_min)
        rating='%.2f'%rating
        return rating
    
lines=[]

for line in f.readlines():
    line=line.replace('\n','')
    new_line=''
    line_split=line.split('|')
    rating=line_split[6]
    user_birthday=line_split[8]
    review_date=line_split[4]
    temperature=line_split[5]
    
    line_split[6]=rating_1(rating)
    line_split[8]=time_std(user_birthday)
    line_split[4]=time_std(review_date)
    line_split[5]=tem_std(temperature)
    
    for word in line_split:
        new_line+=word
        new_line+='|'
    new_line=new_line[:-1]+'\n'
    lines.append(new_line)
f.close()

lines[-1]=lines[-1].replace('\n','')
f=open('./D_Filter.txt','w',encoding='utf-8')
f.writelines(lines)
f.close()

## 数据清洗

In [11]:
'''
review_id|longitude|latitude|altitude|review_date|temperature|rating|user_id|user_birthday|usr_nationality|user_career|usr_income

rating user_income

rating:user_income longitude latitude altitude
user_income:usr_nationality user_career

首先填充user_income选取usr_nationality user_career相同的平均值填充
之后选取rating缺失值的user_income longitude latitude altitude与其他最为接近的rating填充

'''

#首先填充user_income
import numpy as np

f=open('./D_Filter.txt','r',encoding='utf-8')
liens=[]

nc_i_dict={}

Full_lines=[]
Missing_lines=[]
count=0
all_income=0

for line in f.readlines():
    line=line.replace('\n','')
    
    line_split=line.split('|')
    # print(line_split)
    user_income=line_split[-1]
    usr_nationality=line_split[-3]
    user_career=line_split[-2]
    if user_income!='?':
        count+=1
        all_income+=int(user_income)
        Full_lines.append(line+'\n')
        dictkey=usr_nationality+user_career
        if dictkey in nc_i_dict.keys():
            nc_i_dict[dictkey].append(int(user_income))
        else:
            nc_i_dict[dictkey]=[int(user_income)]
    else:
        Missing_lines.append(line)
income_mean=all_income/count

for miss_line in Missing_lines:
    line_split= miss_line.split('|')
    usr_nationality=line_split[-3]
    user_career=line_split[-2]
    dictkey=usr_nationality+user_career
    if dictkey in nc_i_dict.keys():
        line_split[-1]=str(int(np.array(nc_i_dict[dictkey]).mean()))
    else:
        line_split[-1]=str(int(income_mean))
    new_line=''
    for word in line_split:
        new_line+=word
        new_line+='|'
    new_line=new_line[:-1]+'\n'
    Full_lines.append(new_line)

Full_lines[-1]=Full_lines[-1].replace('\n','')

f.close()
f=open('./income_fill.txt','w',encoding='utf-8')
f.writelines(Full_lines)
f.close()
    
    

In [12]:
'''
review_id|longitude|latitude|altitude|review_date|temperature|rating|user_id|user_birthday|usr_nationality|user_career|usr_income

rating user_income

rating:user_income longitude latitude altitude
user_income:usr_nationality user_career

'''
import numpy as np


def nearest_rating(re,values):
    des_n=100000
    rating=0
    for r in re:
        ref_value=r[:-1]
        des=np.sqrt(np.sum((np.array(ref_value)-np.array(values))**2))
        if des<des_n:
            des_n=des
            rating=ref_value[-1]
    return '%.2f'%rating


f=open('./income_fill.txt','r',encoding='utf-8')
liens=[]

Full_lines=[]
Missing_lines=[]

re=[]
#user_income longitude latitude altitude rating

for line in f.readlines():
    line=line.replace('\n','')
    
    
    line_split=line.split('|')
    rating=line_split[6]
    user_income=line_split[-1]
    longitude=line_split[1]
    latitude=line_split[2]
    altitude=line_split[3]
    
    if rating != '?':
        Full_lines.append(line+'\n')
        re.append([int(user_income),float(longitude),float(latitude),float(altitude),float(rating)])
    else:
        Missing_lines.append(line)

f.close()
for line in Missing_lines:
    line_split=line.split('|')
    # rating=line_split[6]
    user_income=line_split[-1]
    longitude=line_split[1]
    latitude=line_split[2]
    altitude=line_split[3]
    line_split[6]=nearest_rating(re,[int(user_income),float(longitude),float(latitude),float(altitude)])
    new_line=''
    for word in line_split:
        new_line+=word
        new_line+='|'
    new_line=new_line[:-1]+'\n'
    Full_lines.append(new_line)

Full_lines[-1]=Full_lines[-1].replace('\n','')

f=open('./D_done.txt','w',encoding='utf-8')
f.writelines(Full_lines)
f.close()
    
    
    